# Final Project PAD

*Author: Jacek Spalinski*

### Goal of the project and data collection

I wanted this project to be not only educational in terms of coding, but also to concern topic interesting to me. I really like Japan and consider living in this country in the future, which will involve renting an apartment there. 

Therefore, I have chosen prediction of apartments' rent in Tokyo as a topic for my project. This will be a linear regression problem, with monthly rent being a decision attribute. Model I create might be later used in real life, should I ever try to rent an apartment in Tokyo.

Data used for this project come from website www.gaijinpot.com. With the help of selenium library I have created small web scraper application, which collected 6000 observations. Data have been loaded into csv file that will be used in this notebook.



### Data cleaning

In [1]:
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH = '/content/drive/MyDrive/PAD/Project/Data' # to be changed depending on working environment

In [4]:
org_df = pd.read_csv(f"{PATH}/apartments_data.csv", index_col=0)
org_df

,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),"Monthly Costs ¥525,000"
1,1DK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),"Monthly Costs ¥288,000"
2,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),"Monthly Costs ¥297,000"
3,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),"Monthly Costs ¥324,000"
4,1K SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),"Monthly Costs ¥225,000"
...,...,...,...,...,...,...,...,...
5995,1K APARTMENT,"IN TSURUMAKI\nSETAGAYA-KU, TOKYO","Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),"Monthly Costs ¥75,000"
5996,1R APARTMENT,"IN KAMIISSHIKI\nEDOGAWA-KU, TOKYO",J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),"Monthly Costs ¥91,540"
5997,1K APARTMENT,"IN KOYAMA\nSHINAGAWA-KU, TOKYO","Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),"Monthly Costs ¥111,500"
5998,1DK APARTMENT,"IN KITAZAWA\nSETAGAYA-KU, TOKYO","DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),"Monthly Costs ¥139,500"


In [5]:
# create copy of original DataFrame, that I will modify
df = org_df.copy()

As we can see, dataset indeed consists of 6000 observations, with 8 attributes (including 1 decision attribute - "cost"). 

First, I will analyse and clean decision attribute.

##### Cost (decision attribute)

We can see by examining DataFrame above, that "cost" attribute should have a form of "Monthly Costs" followed by the amount in JPY. We can't see it, but optionally there is at the end "negotiable". Let's check if all values follow this pattern.

In [6]:
pattern = "Monthly Costs ¥[1-9]+"
mask = [True if re.search(pattern, txt) else False for txt in df["cost"]]
# first let's check values that don't match
opposite_mask = [not x for x in mask]
org_df[opposite_mask].head(5)

,apartment_type,district,agency,size,floor,year_built,station_distance,cost
2919,?,?,Nakagawa Fudosan,Size 19.88 m²,Floor 2 / 3F,Year Built 1990,Kitaayase Station (22 min. walk),"1K APARTMENT\nIN SANO\nADACHI-KU, TOKYO"
2931,?,?,Nakagawa Fudosan,Size 22.00 m²,Floor 3 / 4F,Year Built 1989,"2 min. walk to the nearest bus stop, then 10 m...","1K APARTMENT\nIN KAHEI\nADACHI-KU, TOKYO"
2992,?,?,Nakagawa Fudosan,Size 19.44 m²,Floor 3 / 3F,Year Built 1988,Kitaayase Station (10 min. walk),"1K APARTMENT\nIN YANAKA\nADACHI-KU, TOKYO"
2999,?,?,Nakagawa Fudosan,Size 39.66 m²,Floor 2 / 2F,Year Built 1996,"1 min. walk to the nearest bus stop, then 14 m...",2DK APARTMENT\nIN MINAMIMIZUMOTO\nKATSUSHIKA-K...
3034,?,?,Nakagawa Fudosan,Size 25.20 m²,Floor 1 / 2F,Year Built 1974,Kitaayase Station (10 min. walk),"1DK APARTMENT\nIN OYATA\nADACHI-KU, TOKYO"


We can see there was some error while web scraping in those observations, so they are usless to us and we need to drop them.

In [7]:
df = org_df[mask]
df

,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),"Monthly Costs ¥525,000"
1,1DK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),"Monthly Costs ¥288,000"
2,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),"Monthly Costs ¥297,000"
3,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),"Monthly Costs ¥324,000"
4,1K SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),"Monthly Costs ¥225,000"
...,...,...,...,...,...,...,...,...
5995,1K APARTMENT,"IN TSURUMAKI\nSETAGAYA-KU, TOKYO","Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),"Monthly Costs ¥75,000"
5996,1R APARTMENT,"IN KAMIISSHIKI\nEDOGAWA-KU, TOKYO",J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),"Monthly Costs ¥91,540"
5997,1K APARTMENT,"IN KOYAMA\nSHINAGAWA-KU, TOKYO","Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),"Monthly Costs ¥111,500"
5998,1DK APARTMENT,"IN KITAZAWA\nSETAGAYA-KU, TOKYO","DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),"Monthly Costs ¥139,500"


New DataFrame has 5977, so we droped only 23 observation, which is not significant. Now let's convert string values of this attributes to numbers.

In [8]:
def extract_cost(txt):
  cost = txt.split(" ")[2]
  cost = cost.replace("¥", "")
  cost = cost.replace(",", "")
  return float(cost)

In [9]:
pd.options.mode.chained_assignment = None
df["cost"] = [extract_cost(cost) for cost in df["cost"]]
df

,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...
5995,1K APARTMENT,"IN TSURUMAKI\nSETAGAYA-KU, TOKYO","Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R APARTMENT,"IN KAMIISSHIKI\nEDOGAWA-KU, TOKYO",J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K APARTMENT,"IN KOYAMA\nSHINAGAWA-KU, TOKYO","Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK APARTMENT,"IN KITAZAWA\nSETAGAYA-KU, TOKYO","DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


##### Apartment Type

I marked missing values during web scraping with "?", so let's check if there are any missing values for "apartment_type"

In [10]:
df[df["apartment_type"] == "?"]

,apartment_type,district,agency,size,floor,year_built,station_distance,cost


The result is empty, so we don't have missing values.

In [11]:
df["apartment_type"].value_counts()

1K APARTMENT               3080
1R APARTMENT               1078
1LDK APARTMENT              649
1DK APARTMENT               271
2LDK APARTMENT              270
2DK APARTMENT               209
3LDK APARTMENT              108
2K APARTMENT                 82
3DK APARTMENT                30
3LDK HOUSE                   23
1SLDK APARTMENT              17
2SLDK APARTMENT              17
4LDK APARTMENT               15
3SLDK APARTMENT              12
1SK APARTMENT                12
2SLDK HOUSE                  11
2LDK HOUSE                   10
1LDK HOUSE                    9
1LDK SERVICED APARTMENT       7
SHARED APARTMENT              6
4LDK HOUSE                    6
2DK HOUSE                     6
1K SERVICED APARTMENT         6
3SLDK HOUSE                   5
2LDK SERVICED APARTMENT       4
3DK HOUSE                     4
OFFICE                        3
3LDK TERRACE HOUSE            3
1SDK APARTMENT                3
3K HOUSE                      3
2K HOUSE                      2
4SLDK AP

This variable consist of 2 parts: layout and buidling (apartment) type. I will split it into 2 seperate variables. We have however problem with 3 specific values: SHARED APARTMENT, OFFICE and SHARED GUESTHOUSE, that do not have layout specified. But as they consist of only 11 observation (out of ~6000), and I am not interested in those type of apartments, it is safe to drop them. 

In [12]:
values_to_drop = ["SHARED APARTMENT", "OFFICE", "SHARED GUESTHOUSE"]
df = df[df["apartment_type"].isin(values_to_drop) == False]
df

,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...
5995,1K APARTMENT,"IN TSURUMAKI\nSETAGAYA-KU, TOKYO","Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R APARTMENT,"IN KAMIISSHIKI\nEDOGAWA-KU, TOKYO",J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K APARTMENT,"IN KOYAMA\nSHINAGAWA-KU, TOKYO","Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK APARTMENT,"IN KITAZAWA\nSETAGAYA-KU, TOKYO","DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


And indeed we only removed 11 rows.

In [13]:
layout = [x.split(" ")[0] for x in df["apartment_type"]]
df.insert(0, "layout", layout)
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,1LDK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,1DK SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,1LDK SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,1K SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,1K APARTMENT,"IN TSURUMAKI\nSETAGAYA-KU, TOKYO","Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,1R APARTMENT,"IN KAMIISSHIKI\nEDOGAWA-KU, TOKYO",J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,1K APARTMENT,"IN KOYAMA\nSHINAGAWA-KU, TOKYO","Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,1DK APARTMENT,"IN KITAZAWA\nSETAGAYA-KU, TOKYO","DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


Let's adjust "apartment_type" attribute.

In [14]:
apartment_types = [" ".join(x.split(" ")[1:]) for x in df["apartment_type"]]
df["apartment_type"] = apartment_types
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,"IN NISHIIKEBUKURO\nTOSHIMA-KU, TOKYO",H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,"IN ROPPONGI\nMINATO-KU, TOKYO",H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,"IN TSURUMAKI\nSETAGAYA-KU, TOKYO","Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,"IN KAMIISSHIKI\nEDOGAWA-KU, TOKYO",J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,"IN KOYAMA\nSHINAGAWA-KU, TOKYO","Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,"IN KITAZAWA\nSETAGAYA-KU, TOKYO","DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


##### District

In this variable we are only interested in district, which is a word ending with "-ku". 

In [15]:
df[df["district"] == "?"]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost


No missing values.

In [16]:
def extract_distict(txt):
  txt = txt.replace("\n", " ")
  district = txt.split(" ")[2]
  district = district.replace(",", "")
  return district

In [17]:
df["district"] = [extract_distict(district) for district in df["district"]]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


In [18]:
df["district"].value_counts()

SETAGAYA-KU      651
MINATO-KU        485
OTA-KU           463
SHINJUKU-KU      391
NERIMA-KU        372
SHIBUYA-KU       327
ADACHI-KU        322
MEGURO-KU        291
SHINAGAWA-KU     261
ITABASHI-KU      259
TOSHIMA-KU       255
SUGINAMI-KU      239
EDOGAWA-KU       222
NAKANO-KU        196
KOTO-KU          176
SUMIDA-KU        167
KATSUSHIKA-KU    157
TAITO-KU         156
KITA-KU          134
BUNKYO-KU        108
ARAKAWA-KU       105
CHUO-KU           94
CHIYODA-KU        45
(2-4-CHOME)       18
TSURUMAKICHO      11
MIDORICHO          6
NISHIMACHI         5
(1-CHOME)          4
KAWARACHO          4
KANAICHO           3
NAKAICHO           3
TATSUTACHO         3
ASAHICHO           3
(3-CHOME           2
SAKURAGI           2
HIGASHIMACHI       2
HASHIDOCHO         2
MINAMIMACHI        2
KITACHO            2
OKAWACHO           2
KAMICHO            2
AZUMA              2
NAKACHO            2
KOTOBUKICHO        2
HIGASHICHO         2
MIYAMOTOCHO        1
NISHICHO           1
YANAGICHO    

I expected 23 values, as core Tokyo has 23 districts, but there are also some different values. I think those values are sort of sub-districts, but to be honest I will not make an effort and check which districts they correspond to. The amount of those observations is quite small, so I can safely just delete them.

In [19]:
pattern = "[A-Z]+-KU"
mask = [True if re.search(pattern, txt) else False for txt in df["district"]]
df = df[mask]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,Size 50.50 m²,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,Size 29.20 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,Size 35.40 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,Size 37.90 m²,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,Size 25.40 m²,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",Size 17.86 m²,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,Size 27.72 m²,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",Size 32.12 m²,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",Size 30.97 m²,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


This operation deleted 90 observations, which was around 1,5% of the dataset, so it is fine. Let's also verify that now this column consists of only 23 unique values.

In [20]:
len(df["district"].unique())

23

##### Agency

Basically all aparemnts for rent in Japan are offered through an agency, and agencies might vary in their margins, which might affect the proce (decision attribute).

In [21]:
df["agency"].value_counts()

Aonissin Co.,ltd.                                                                 1824
Sumaino Seika                                                                     1354
DAITOKENTAKU LEASING CO.,LTD.                                                      411
Atinn Inc.                                                                         359
Balleggs Co., Ltd.                                                                 345
J-Homes                                                                            256
Tokyo Comfort Inc.                                                                 220
matsuri technologies                                                               179
Century 21 Nishin Co., Ltd.                                                        166
World Clover                                                                       131
Yours Corporation Co.,Ltd.                                                         101
World Potential Corporation                

"Add Value" was a bit suspicious, but I checked and this is indeed a name of an agency, so nothing to do with this attribute.

##### Size

In [22]:
df[df["size"] == "?"]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost


No missing values.

The "size" attribute should follow "Size {number} m^2" pattern.

In [23]:
pattern = "Size [1-9][0-9]?[0-9]?\.[0-9][0-9] m²"
mask = [True if re.search(pattern, txt) else False for txt in df["size"]]
# first let's check values that don't match
opposite_mask = [not x for x in mask]
df[opposite_mask].head(5)

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost


All values follow the pattern. Now let's convert this attribute to a numerical one.

In [24]:
def extract_size(txt):
  size = txt.split(" ")[1]
  return float(size)

In [25]:
df["size"] = [extract_size(size) for size in df["size"]]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


##### Floor

In [26]:
df[df["floor"] == "?"]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost


No missing values.

The "floor" attribute should follow "Floor {number} / {number}F" pattern. Also, to avoid any confusion with the regex, in Japan ground floor is floor 1, so you can't have floor 0.

In [27]:
pattern = "Floor [1-9][0-9]? / [1-9][0-9]?F"
mask = [True if re.search(pattern, txt) else False for txt in df["floor"]]
# first let's check values that don't match
opposite_mask = [not x for x in mask]
df[opposite_mask]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
114,3LDK,HOUSE,NAKANO-KU,"Aonissin Co.,ltd.",69.56,Year Built 2020,Nearest Station Tamachi Station (5 min. walk),?,210000.0
148,2LDK,HOUSE,MINATO-KU,"Aonissin Co.,ltd.",53.75,Year Built 1966,Nearest Station Omotesando Station (8 min. walk),?,190000.0
194,2LDK,HOUSE,TOSHIMA-KU,"Aonissin Co.,ltd.",80.59,Year Built 1978,Nearest Station Shinotsuka Station (8 min. walk),?,190000.0
224,1LDK,HOUSE,SETAGAYA-KU,"Aonissin Co.,ltd.",60.39,Year Built 2010,Nearest Station Kaminoge Station (9 min. walk),?,170000.0
237,1LDK,HOUSE,EDOGAWA-KU,"Aonissin Co.,ltd.",45.13,Year Built 1973,Nearest Station Shinozaki Station (8 min. walk),?,93000.0
...,...,...,...,...,...,...,...,...,...
5357,1LDK,APARTMENT,OTA-KU,"Balleggs Co., Ltd.",47.10,Floor B1 / 5F,Year Built 1991,Ishikawadai Station (9 min. walk),140000.0
5413,2SLDK,HOUSE,BUNKYO-KU,"Century 21 Nishin Co., Ltd.",83.61,Year Built 1993,Nearest Station Hakusan Station (5 min. walk),?,270110.0
5443,3LDK,HOUSE,SETAGAYA-KU,"Century 21 Nishin Co., Ltd.",81.90,Year Built 2015,Nearest Station Chitosefunabashi Station (20 m...,?,280000.0
5471,3LDK,HOUSE,SHIBUYA-KU,PIPI Hosting,78.00,Year Built 2015,Nearest Station Nishishinjuku Go Choume Statio...,?,410000.0


We have 84 observations not following the pattern. However, something strange is happening there, I will investigate a bit further.

In [28]:
df[opposite_mask]["floor"].value_counts()

Year Built 2015    8
Year Built 2020    5
Year Built 2016    5
Year Built 2001    5
Year Built 2003    4
Year Built 1992    3
Year Built 2002    2
Year Built 1988    2
Year Built 2011    2
Year Built 1968    2
Year Built 2021    2
Year Built 2018    2
Year Built 2017    2
Year Built 1990    2
Year Built 2012    2
Floor B1 / 5F      2
Year Built 2019    2
Year Built 1995    2
Year Built 1999    2
Year Built 1997    2
Year Built 2010    2
Year Built 1994    2
Year Built 2008    1
Year Built 1985    1
Year Built 1993    1
Year Built 1978    1
Floor B9 / 12F     1
Floor B1 / 15F     1
Year Built 1983    1
Year Built 2009    1
Year Built 1969    1
Year Built 1972    1
Year Built 1959    1
Year Built 2005    1
Year Built 1973    1
Year Built 1998    1
Year Built 2000    1
Year Built 1981    1
Year Built 1989    1
Year Built 2022    1
Year Built 1982    1
Year Built 2013    1
Year Built 1966    1
Year Built 1975    1
Name: floor, dtype: int64

There are 4 values that start with "B". It might mean basement, but I coundn't find information about that, and what B9 would mean then? I will just drop those values. 

More interesting are values "Year Built", as they belong to "year_built" attribute. What happened here, attributes on the website were stored in the list and didn't have a unique id's. Os here instead of missing value it just took next attribute on the list, which is year built. Also, as we saw, for those observations "year_built" column consist of values for "station_distance" attribute. So in reality those observations contain missing values for floor. 

I could try to substitute those values with for examle mean, but as it's again only 84 observations in total (less than 1,5% of the dataset), I will just drop it.

In [29]:
df = df[mask]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,Floor 11 / 14F,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,Floor 16 / 26F,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,Floor 3 / 14F,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,Floor 2 / 3F,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,Floor 2 / 2F,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,Floor 1 / 3F,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,Floor 2 / 3F,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


Now I will convert those values just to the floor number.

In [30]:
def extract_floor(txt):
  floor = txt.split(" ")[1]
  return int(floor)

In [31]:
df["floor"] = [extract_floor(floor) for floor in df["floor"]]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,11,Year Built 2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,3,Year Built 2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,16,Year Built 2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,16,Year Built 2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,3,Year Built 2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,Year Built 1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,2,Year Built 2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,Year Built 2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,Year Built 2020,Sasazuka Station (8 min. walk),139500.0


##### Year Built

In [32]:
df[df["year_built"] == "?"]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost


No missing values (but after "floor" attribute we know it is not necessarily true).

The "year_built" attribute should follow "Year Built {number}" pattern.

In [33]:
pattern = "Year Built [1-2][0-9]{3}"
mask = [True if re.search(pattern, txt) else False for txt in df["year_built"]]
# first let's check values that don't match
opposite_mask = [not x for x in mask]
df[opposite_mask]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost


It seems all values follow the pattern. Let's now convert it just to year value.

In [34]:
def extract_year(txt):
  year = txt.split(" ")[2]
  return int(year)

In [35]:
df["year_built"] = [extract_year(year) for year in df["year_built"]]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,11,2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,3,2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,16,2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,16,2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,3,2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,2,2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,Sasazuka Station (8 min. walk),139500.0


##### Station Distance

In [36]:
df[df["station_distance"] == "?"]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
37,1R,APARTMENT,SETAGAYA-KU,"MINAMIAOYAMA REAL ESTATE Co., Ltd",16.50,3,1987,?,70000.0
3040,1R,APARTMENT,ITABASHI-KU,"Aonissin Co.,ltd.",18.00,2,1991,?,62000.0
3249,1R,APARTMENT,ITABASHI-KU,"Miyoshi Real Estate Co.,Ltd.",7.00,1,2017,?,48000.0
3934,1LDK,APARTMENT,MINATO-KU,matsuri technologies,41.70,13,2007,?,233400.0
4135,1R,APARTMENT,MINATO-KU,matsuri technologies,25.50,4,2007,?,163500.0
4138,1LDK,APARTMENT,MEGURO-KU,"Yours Corporation Co.,Ltd.",32.13,9,1966,?,130000.0
4493,1R,APARTMENT,MINATO-KU,matsuri technologies,25.50,2,2007,?,163500.0
4499,1LDK,APARTMENT,MINATO-KU,matsuri technologies,41.70,12,2007,?,233400.0
4715,1LDK,APARTMENT,MEGURO-KU,Tokyo Comfort Inc.,34.94,5,2002,?,171000.0
5506,1LDK,APARTMENT,CHIYODA-KU,RE/MAX Amistad,53.28,5,2006,?,293000.0


There are some missing values, but there are so few I can safely drop them.

In [37]:
df = df[df["station_distance"] != "?"]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,11,2007,Roppongi Station (6 min. walk),525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,3,2007,Roppongi Station (6 min. walk),288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,16,2007,Ikebukuro Station (2 min. walk),297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,16,2007,Ikebukuro Station (2 min. walk),324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,3,2007,Roppongi Station (6 min. walk),225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,Sakurashinmachi Station (9 min. walk),75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,2,2015,Shinkoiwa Station (17 min. walk),91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,Nishikoyama Station (3 min. walk),111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,Sasazuka Station (8 min. walk),139500.0


It seems the "year_built" attribute should follow "{Station Name} Station ({number} min. walk/by bus)" pattern.

In [38]:
pattern = ".+ Station [(][1-9][0-9]? min.+[)]"
mask = [True if re.search(pattern, txt) else False for txt in df["station_distance"]]
# first let's check values that don't match
opposite_mask = [not x for x in mask]
df[opposite_mask]

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
1043,1K,APARTMENT,EDOGAWA-KU,Sumaino Seika,20.28,2,2005,"3 min. walk to 東小松川小学校前 bus stop, then 16 min....",65550.0
1557,1K,APARTMENT,EDOGAWA-KU,Sumaino Seika,19.87,2,2005,"8 min. walk to the nearest bus stop, then 8 mi...",72050.0
1852,1K,APARTMENT,SUGINAMI-KU,Sumaino Seika,20.28,1,2006,"4 min. walk to 桃井４丁目 bus stop, then 16 min. by...",69550.0
1875,1K,APARTMENT,NERIMA-KU,Sumaino Seika,19.87,2,1999,"9 min. walk to 3 bus stop, then 28 min. by bus...",56550.0
1943,1K,APARTMENT,KATSUSHIKA-KU,Sumaino Seika,20.83,1,2007,"2 min. walk to 大谷田５丁目 bus stop, then 20 min. b...",68050.0
1955,1K,APARTMENT,SETAGAYA-KU,Sumaino Seika,19.87,1,2010,"3 min. walk to 砧本村 bus stop, then 20 min. by b...",70550.0
1988,1K,APARTMENT,EDOGAWA-KU,Sumaino Seika,19.87,1,2003,"3 min. walk to 一之江橋西詰 bus stop, then 15 min. b...",65550.0
2082,1K,APARTMENT,EDOGAWA-KU,Sumaino Seika,19.87,2,2006,"7 min. walk to 一之江橋西詰 bus stop, then 15 min. b...",60550.0
2102,1K,APARTMENT,NERIMA-KU,Sumaino Seika,19.87,1,2002,"9 min. walk to 3 bus stop, then 28 min. by bus...",54550.0
2158,1K,APARTMENT,EDOGAWA-KU,Sumaino Seika,19.08,1,2003,"3 min. walk to the nearest bus stop, then 9 mi...",61550.0


I see another pattern, where there is some time to bus stop, and then some time by bus to subway station. However, those values are so schematic I can extract those values and just add them. It might underestimate the distance, as it does not include waiting time for a bus, but I decide to accept this limitation.

In [39]:
def extract_distance_transfer(txt):
  parts = txt.split("then")
  dist1 = parts[0].split(" ")[0]
  dist2 = parts[1].split(" ")[1]
  return int(dist1) + int(dist2)

And function for normal cases.

In [40]:
def extract_distance(txt):
  txt = txt.replace("(", "")
  dists = txt.split(" ")
  # Station name can vary in length, so I cannot just select given index
  for dist in dists:
    if dist.isdigit():
      return int(dist)
  

In [41]:
df["station_distance"][mask] =  [extract_distance(dist) for dist in df["station_distance"][mask]]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,11,2007,6,525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,3,2007,6,288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,16,2007,2,297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,16,2007,2,324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,3,2007,6,225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,9,75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,2,2015,17,91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,3,111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,8,139500.0


In [42]:
df["station_distance"][opposite_mask] = [extract_distance_transfer(dist) for dist in df["station_distance"][opposite_mask]]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
0,1LDK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,50.50,11,2007,6,525000.0
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,3,2007,6,288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,16,2007,2,297000.0
3,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,37.90,16,2007,2,324000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,3,2007,6,225000.0
...,...,...,...,...,...,...,...,...,...
5995,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,9,75000.0
5996,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,2,2015,17,91540.0
5997,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,3,111500.0
5998,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,8,139500.0


Let's just double check we got all correct values.

In [43]:
df["station_distance"].value_counts()

5     645
8     571
6     553
7     539
4     469
3     447
9     441
10    414
2     312
12    218
1     199
11    195
13    158
15    153
14    112
16     90
18     54
17     51
19     44
20     32
22     21
21     16
23     13
24      8
28      5
25      3
27      3
29      3
37      2
26      2
32      1
33      1
38      1
31      1
Name: station_distance, dtype: int64

Last sanity check if all attributes have correct data types.

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5777 entries, 0 to 5999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   layout            5777 non-null   object 
 1   apartment_type    5777 non-null   object 
 2   district          5777 non-null   object 
 3   agency            5777 non-null   object 
 4   size              5777 non-null   float64
 5   floor             5777 non-null   int64  
 6   year_built        5777 non-null   int64  
 7   station_distance  5777 non-null   object 
 8   cost              5777 non-null   float64
dtypes: float64(2), int64(2), object(5)
memory usage: 451.3+ KB


And we see "station_distance" is still an object, let's change it to int.

In [45]:
df["station_distance"] = df["station_distance"].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5777 entries, 0 to 5999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   layout            5777 non-null   object 
 1   apartment_type    5777 non-null   object 
 2   district          5777 non-null   object 
 3   agency            5777 non-null   object 
 4   size              5777 non-null   float64
 5   floor             5777 non-null   int64  
 6   year_built        5777 non-null   int64  
 7   station_distance  5777 non-null   int64  
 8   cost              5777 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 451.3+ KB


Finally, let's reset the index.

In [46]:
df.reset_index(drop=True, inplace=True)

### Data visualization and analysis

##### Correlation

In [47]:
import plotly.express as px

In [48]:
fig = px.imshow(df.corr(), text_auto=True, aspect="auto")
fig.update_layout(title="Correlation matrix")
fig.show()

Looking at correlation matrix, not surprisingly we can spot that cost is highly correlated with the size of an apartment. There is also positive (although not that strong) correlation between cost and floor and year_built, which fits the logic that people prefer apartments higher in the building and in the newer buildings, hence the higher price for such apartments. On the other hand station_distance is negatively correlated, which also makes sense, as the further station is, the less attractive apartment is.

We can also see no predictive attributes are highly correlated with each other, so we can use potentially use all of them in the regression model. There are however some interesting observations:

*   There is positive correlation between size and floor, suggesting bigger apartments are built higher in the building - or taller buildings have bigger apartments in general.
*   There is positive correltaion between year_built and station_distance, that might suggest as time pogresses, most pieces of land close to the stations are already taken.
*   Negative correlation between size and year_built might be explained by growing demand for apartments for rents and as time go by, developers build smaller apartments, in order to fit more of the on a given piece of land.

Of course I could analyze each correlation, but I do not think there are other relations that have some "story" behind them.

##### Cost relations and visualizations

First let's look for outliers in decision attribute.

In [49]:
fig = px.box(df, y="cost", title="Box plot of cost attribute")
fig.show()

We can see we have some observations that should be considered outliers (Q3 + 1,5*IQR ~= 210k). Let's check how many observations can be considered outliers.

In [50]:
len(df[df["cost"] > 210110])

493

It's quite a substantial amount, as 493 observations are around 8,5% of dataset we are currently working with. I will be a bit more conservative for now and start with 300k as a treshold.

In [51]:
len(df[df["cost"] > 300000])

222

Now it's only 22 observations, so let's remove them and check the box plot again.

In [52]:
df = df[df["cost"] <= 300000]
df

,layout,apartment_type,district,agency,size,floor,year_built,station_distance,cost
1,1DK,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,29.20,3,2007,6,288000.0
2,1LDK,SERVICED APARTMENT,TOSHIMA-KU,H2O Japan Property,35.40,16,2007,2,297000.0
4,1K,SERVICED APARTMENT,MINATO-KU,H2O Japan Property,25.40,3,2007,6,225000.0
5,1K,APARTMENT,MINATO-KU,"HASEKO LIVENET, Inc.",22.00,4,2018,3,255000.0
6,1K,APARTMENT,SHIBUYA-KU,NOW ROOM,17.00,4,1990,5,130200.0
...,...,...,...,...,...,...,...,...,...
5772,1K,APARTMENT,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,9,75000.0
5773,1R,APARTMENT,EDOGAWA-KU,J-Homes,27.72,2,2015,17,91540.0
5774,1K,APARTMENT,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,3,111500.0
5775,1DK,APARTMENT,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,8,139500.0


In [53]:
fig = px.box(df, y="cost", title="Box plot of cost attribute after outliers removal")
fig.show()

We obviously still have some observations that might be considered outliers, but I will argue 300k is not so far away from other values and I will keep it.

Now with the largest outliers removed, let's have a look at distribution of the decision attribute.

In [54]:
fig = px.histogram(df, 
                   x="cost",
                   title="Counts of apartment rental prices")
fig.show()

We can say it is roughly normal, though histogram has a fat right tail, making distribution right-skewed.

Let's check which districts are the most expensive.

In [55]:
district_prices = df[["district", "cost"]].groupby("district").mean().round(0)
district_prices.reset_index(inplace=True)

In [56]:
fig = px.histogram(district_prices,
                   x="district",
                   y="cost",
                   title="Average cost by district",
                   labels={"district": "district", "cost": "average cost"})
fig.update_layout(xaxis={'categoryorder': 'total descending'}, yaxis_title="average cost")
fig.show()

We can see that the most expensive districts are Minato, Chuo, Chiyoda, Shibuya and Shinjuku. Looking at the map of Tokyo it makes sense, as those are all central districts of Tokyo. Let's now look at relation between "cost" and "apartment_type".

In [57]:
df["apartment_type"].value_counts()

APARTMENT             5540
SERVICED APARTMENT       8
TERRACE HOUSE            4
HOUSE                    3
Name: apartment_type, dtype: int64

As we see, most of the apartment are of the same type - APARTMENT. Let's see the relation of different types to the costs.

In [58]:
fig = px.box(df, x="apartment_type", y="cost",
             title="Cost based on apartment type")
fig.show()

SERVICED APARTMENT and TERRACE HOUSE are significantly more expensive, in the range to be classified as outliers. HOUSE on the other hand is still more expensive than APARTMENT, but within Q3 + 1,5*IQR range. I think I will delete observations that are not APARTMENTS, and then, when I'm left with only one value, I can reduce dimensionality of the dataset and drop this attribute altogether.

In [59]:
df = df[df["apartment_type"] == "APARTMENT"]
df.drop(columns="apartment_type", inplace=True)
df

,layout,district,agency,size,floor,year_built,station_distance,cost
5,1K,MINATO-KU,"HASEKO LIVENET, Inc.",22.00,4,2018,3,255000.0
6,1K,SHIBUYA-KU,NOW ROOM,17.00,4,1990,5,130200.0
7,1R,SHINJUKU-KU,NOW ROOM,30.00,10,1989,3,175000.0
8,1K,TAITO-KU,matsuri technologies,20.15,2,2019,12,139500.0
9,2DK,NAKANO-KU,Fontana,40.00,5,1978,4,152000.0
...,...,...,...,...,...,...,...,...
5772,1K,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,9,75000.0
5773,1R,EDOGAWA-KU,J-Homes,27.72,2,2015,17,91540.0
5774,1K,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,3,111500.0
5775,1DK,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,8,139500.0


Finally let's explore visually relation between "size" and "cost", as we identified strong correlation between those attributes earlier. 

In [60]:
fig = px.scatter(df, x="size", y="cost", trendline="ols",
                 title="Cost based on size of the apartment")
fig.show()

Confirming what we saw earlier, there is visible positive trend - as size rises, so does the cost.

While we analyse "size", let's add third dimension to the graph - "layout". I expect layout to be highly related with size, as it makes sense e.g. 1 room apartment is smaller than 2 room apartment.

In [61]:
fig = px.scatter(df, x="size", y="cost", color="layout",
                 title="Cost based on size and layout of the apartment")
fig.show()

Just by looking at the graph we can observe clusters of apartments with the same layout, but I think we should investigate this relation a bit further.

##### Other relations and visualizations

Let's check average size of the apartment based on it's layout. 

In [62]:
layout_sizes = df[["layout", "size"]].groupby("layout").mean().round(2)
layout_sizes.reset_index(inplace=True)

In [63]:
fig = px.histogram(layout_sizes,
                   x="layout",
                   y="size",
                   title="Average size of apartment by its layout",
                   labels={"layout": "layout", "size": "average size"})
fig.update_layout(xaxis={'categoryorder': 'total ascending'}, yaxis_title="average size")
fig.show()

I will not go to deep into interpreting this graph. Using categorical variables is somehow difficult, as you have to use one-hot encoding and you end up with a lot of binary variables. For that reason I will not use "layout" attribute in my model, as I strongly believe "size" by itself has enough information.

In [64]:
df.drop(columns="layout", inplace=True)
df

,district,agency,size,floor,year_built,station_distance,cost
5,MINATO-KU,"HASEKO LIVENET, Inc.",22.00,4,2018,3,255000.0
6,SHIBUYA-KU,NOW ROOM,17.00,4,1990,5,130200.0
7,SHINJUKU-KU,NOW ROOM,30.00,10,1989,3,175000.0
8,TAITO-KU,matsuri technologies,20.15,2,2019,12,139500.0
9,NAKANO-KU,Fontana,40.00,5,1978,4,152000.0
...,...,...,...,...,...,...,...
5772,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,9,75000.0
5773,EDOGAWA-KU,J-Homes,27.72,2,2015,17,91540.0
5774,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,3,111500.0
5775,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,8,139500.0


As we removed one categorical variable, we should explore the other one - "agency". As discussed, I expect this attribute to be important, as different agencies might have different margins. But let's check it. However, in order to limit the number of potential binary variables, I will exclude agencies that have less than 20 offers.

In [65]:
agencies = [agency for agency in df["agency"].unique() if len(df[df["agency"] == agency]) >= 20]
print(f"{len(df['agency'].unique()) - len(agencies)} agencies have less than 20 offers")

12 agencies have less than 20 offers


In [66]:
df = df[df["agency"].isin(agencies) == True]
df

,district,agency,size,floor,year_built,station_distance,cost
6,SHIBUYA-KU,NOW ROOM,17.00,4,1990,5,130200.0
7,SHINJUKU-KU,NOW ROOM,30.00,10,1989,3,175000.0
8,TAITO-KU,matsuri technologies,20.15,2,2019,12,139500.0
9,NAKANO-KU,Fontana,40.00,5,1978,4,152000.0
10,SHIBUYA-KU,NOW ROOM,20.00,4,2000,4,165000.0
...,...,...,...,...,...,...,...
5772,SETAGAYA-KU,"Yours Corporation Co.,Ltd.",17.86,2,1993,9,75000.0
5773,EDOGAWA-KU,J-Homes,27.72,2,2015,17,91540.0
5774,SHINAGAWA-KU,"Balleggs Co., Ltd.",32.12,1,2010,3,111500.0
5775,SETAGAYA-KU,"DAITOKENTAKU LEASING CO.,LTD.",30.97,2,2020,8,139500.0


In [67]:
import numpy as np
import plotly.graph_objects as go

In [68]:
N = len(agencies)
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
fig = go.Figure(data=[go.Box(
    name=agencies[i],
    y=df[df["agency"] == agencies[i]]["cost"],
    marker_color=c[i]
    ) for i in range(int(N))])
fig.update_layout(
    xaxis=dict(showticklabels=False, title="Agency"),
    yaxis=dict(title="Cost"),
    title="Cost by agency"
)
fig.show()

We can see that the cost indeed vary based on the agencies. However, it might be the case some agencies just have bigger apartments, hence higher costs. Let's see.

In [69]:
fig = go.Figure(data=[go.Box(
    name=agencies[i],
    y=df[df["agency"] == agencies[i]]["size"],
    marker_color=c[i]
    ) for i in range(int(N))])
fig.update_layout(
    xaxis=dict(showticklabels=False, title="Agency"),
    yaxis=dict(title="Size"),
    title="Size by agency"
)
fig.show()

Looking at 2 graphs, their shape is somehow similar, however:

1.   There are some differences, e.g. "Tokyo Monthly" have higher prices than "J-Homes", even though sizes of offered apartments are similar
2.   Differences in costs seem to have bigger magnitude than in sizes

For those reasons I decide to keep this attribute.



### Regression

##### Model

Now we can proceed to creating our regression model. First I need to split data in training and test sets. I will use typical 70/30 proportion.

In [70]:
from sklearn.model_selection import train_test_split

In [71]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns="cost"),
    df["cost"],
    test_size=0.3,
    random_state=42 
)

And for the model itself.

In [72]:
import statsmodels.formula.api as smf

In [73]:
model = smf.ols(formula='cost ~ C(district) + C(agency) + size + floor + year_built + station_distance', data=x_train.assign(cost=y_train)).fit()

##### Interpretation and test

In [74]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   cost   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     579.5
Date:                Thu, 06 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:06:23   Log-Likelihood:                -42892.
No. Observations:                3846   AIC:                         8.588e+04
Df Residuals:                    3799   BIC:                         8.617e+04
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

First thing to notice is relatively high R-squared of 0,875. Also all numerical attributes are statistically significant, with p-values < 0,05. Some of binary variables derived from categorical attributes have p-values > 0,05, but we can't interpret their significance individually. 

As for coefficient, we have negative intercept, which means we "start" with negative cost and other attributes add to the cost. 

Looking at districts, we can see that indeed the highest coefficient have districts previously identified as the most expensive. The most expensive district - Minato - adds more than 50000 JPY to the rental price. 

As for agancies, we indeed can see variation in coefficient values, which validates our hypothesis that different agencies might have different margins.

Each extra square meter of size add around 2500 JPY to cost, each floor higher adds just more than 1300 JPY, while each year in year built adds almost 900 JPY (meaning the newer the apartment, the higher the price). On the other hand each minute to nearest stations lowers the cost by more than 550 JPY. 

Finally, let's test our model on test dataset and see what is a mean squared error and mean absolute error.

In [75]:
y_pred = model.predict(x_test)

In [76]:
from sklearn.metrics import mean_squared_error

In [80]:
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse}")

MSE: 288960604.2457463


However, more meaningful metric for human is mean absolute error.

In [78]:
from sklearn.metrics import mean_absolute_error

In [82]:
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae}")

MAE: 11516.975878370426


So we see our model on average was off by around 11500 JPY.